In [2]:
# Librería para conectarme a PostgreSQL
import psycopg2    
import pandas as pd

# Creo mi base de datos en Render y me da estas credenciales para conectarme
host = "dpg-cl12f72s1bgc73e1chg0-a.frankfurt-postgres.render.com"
port = 5432
database = "proyecto_thebridge"
user = "proyecto_thebridge_user"
password = "lTs9oLCPsDp9ujNKqcjc3x6Mkhnso9xh"


In [3]:
# Intentamos conectar a la base de datos
conn = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password
)
print("Conexión exitosa a la base de datos en Render")

Conexión exitosa a la base de datos en Render


In [89]:
# Una vez conectada, creamos la tabla
cursor = conn.cursor()

cursor.execute("""
            CREATE TABLE "Profesores" (
                "Id" SERIAL PRIMARY KEY,
                "Nombre" VARCHAR(255) NOT NULL,
                "Vertical" VARCHAR(255) NOT NULL,
                "Rol" VARCHAR(255) NOT NULL,
                "Promoción" VARCHAR(255) NOT NULL,
                "Id_Campus" VARCHAR(255) NOT NULL)
        """)
# Confirmamos la creación de la tabla. El commit es importante porque la Base de datos no está en local
conn.commit()

# Cerramos la conexión
conn.close()

In [39]:
# Cargamos el DF

df = pd.read_csv('data/vertical.csv')    
df = df.drop_duplicates()

In [42]:
df['Campus'] = df['Campus'].apply(lambda x: x.replace("Madrid","1"))
df['Campus'] = df['Campus'].apply(lambda x: x.replace("Valencia","2"))

In [43]:
df['Campus'] = df['Campus'].astype('Int64')

In [51]:
df_profesores = pd.read_csv('data/verticales_2.csv', index_col=0)

In [53]:
df_profesores.columns

Index(['Nombre', 'Rol', 'vertical_id', 'rama', 'promocion'], dtype='object')

In [58]:
alumnos = pd.read_csv('data/alumnos.csv', index_col=0)

In [65]:
alumnos.head()

,Nombre,Email,vertical_id
0,Jafet Casals,Jafet_Casals@gmail.com,55
2,Jorge Manzanares,Jorge_Manzanares@gmail.com,55
4,Onofre Adadia,Onofre_Adadia@gmail.com,55
6,Merche Prada,Merche_Prada@gmail.com,55
8,Pilar Abella,Pilar_Abella@gmail.com,55


ForeignKeyViolation: insert or update on table "alumno" violates foreign key constraint "alumno_alumno_id_fkey"
DETAIL:  Key (alumno_id)=(4) is not present in table "expedientes_ds".


In [72]:
listado_ds_1 = pd.read_csv('data/listado_DS_clase_1.csv', sep=";")

In [83]:
listado_ds_2 = pd.read_csv('data/listado_DS_clase_2.csv', sep=";")

In [82]:
listado_ds_1 = listado_ds_1.drop(columns=['Email', 'Promoción', 'Fecha_comienzo', 'Campus'])

In [84]:
listado_ds_2 = listado_ds_2.drop(columns=['Email', 'Promoción', 'Fecha_comienzo', 'Campus'])

In [87]:
alumnos_ds = pd.concat([listado_ds_1, listado_ds_2])

In [88]:
alumnos_ds.columns

Index(['Nombre', 'Proyecto_HLF', 'Proyecto_EDA', 'Proyecto_BBDD',
       'Proyecto_ML', 'Proyecto_Deployment'],
      dtype='object')

In [91]:

# Volvemos a crear la conexión 

conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )

cursor = conn.cursor()

# Iteramos por las filas y metemos los datos del Df en las columnas correspondientes
for index, row in alumnos_ds.iterrows():
    cursor.execute("""
        INSERT INTO "expedientes_ds" ("proyecto_hlf", "proyecto_eda", "proyecto_bbdd", "proyecto_ml", "proyecto_deployment")
        VALUES (%s, %s, %s, %s, %s)
    """, (row['Proyecto_HLF'], row['Proyecto_EDA'], row['Proyecto_BBDD'], row['Proyecto_ML'], row['Proyecto_Deployment']))
conn.commit()
# cursor.close()

In [99]:
listado_fs_1.columns

Index(['Nombre', 'Email', 'Promoción', 'Fecha_comienzo', 'Campus',
       'Proyecto_WebDev', 'Proyecto_FrontEnd', 'Proyecto_Backend',
       'Proyecto_React', 'Proyecto_FullSatck'],
      dtype='object')

In [102]:
listado_fs_1 = pd.read_csv('data/listado_FS_clase_1.csv', sep=";")
listado_fs_2 = pd.read_csv('data/listado_FS_clase_2.csv', sep=";")
listado_fs_1 = listado_fs_1.drop(columns=['Email', 'Promoción', 'Fecha_comienzo', 'Campus'])
listado_fs_2 = listado_fs_2.drop(columns=['Email', 'Promoción', 'Fecha_comienzo', 'Campus'])
alumnos_fs = pd.concat([listado_fs_1, listado_fs_2])

In [106]:
alumnos_fs.columns

Index(['Nombre', 'Proyecto_WebDev', 'Proyecto_FrontEnd', 'Proyecto_Backend',
       'Proyecto_React', 'Proyecto_FullSatck'],
      dtype='object')

In [109]:

# Volvemos a crear la conexión 

conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )

cursor = conn.cursor()

# Iteramos por las filas y metemos los datos del Df en las columnas correspondientes
for index, row in alumnos_fs.iterrows():
    cursor.execute("""
        INSERT INTO "expediente_fs" ("proyecto_webdev", "proyecto_frontend", "proyecto_backend", "proyecto_react", "proyecto_fullstack")
        VALUES (%s, %s, %s, %s, %s)
    """, (row['Proyecto_WebDev'], row['Proyecto_FrontEnd'], row['Proyecto_Backend'], row['Proyecto_React'], row['Proyecto_FullSatck']))
conn.commit()
# cursor.close()

In [113]:
# Para hacer consulta de los datos subidos a SQL
conn = psycopg2.connect(
            host=host,
            port=port,
            database=database,
            user=user,
            password=password
        )
cursor = conn.cursor()

cursor.execute('SELECT * FROM "expediente_fs"')

# Recupera los resultados de la consulta
rows = cursor.fetchall()

cursor.execute('SELECT * FROM "expedientes_ds"')
# Me lo muestra como un DF
rows2 = cursor.fetchall()


In [115]:
alumnos_ds_query = pd.DataFrame(data=rows2)
alumnos_fs_query = pd.DataFrame( data=rows)

In [118]:
alumnos_ds_query = alumnos_ds_query.drop(columns=[1,2,3,4,5])
alumnos_fs_query = alumnos_fs_query.drop(columns=[1,2,3,4,5])

In [120]:
alumnos_concat = pd.concat([alumnos_ds_query, alumnos_fs_query])

In [123]:
alumnos_concat.shape

(52, 1)

In [130]:
alumnos_concat.columns

RangeIndex(start=0, stop=1, step=1)

In [150]:
alumnos_concat.values

array([[ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19],
       [20],
       [21],
       [22],
       [23],
       [24],
       [25],
       [26],
       [27],
       [28],
       [29],
       [30],
       [31],
       [32],
       [33],
       [34],
       [35],
       [36],
       [37],
       [38],
       [39],
       [40],
       [41],
       [42],
       [43],
       [44],
       [45],
       [46],
       [47],
       [48],
       [49],
       [50],
       [51],
       [52],
       [53],
       [54],
       [55],
       [56]], dtype=int64)

In [151]:
alumnos['alumnos_id'] = alumnos_concat.values

In [161]:
alumnos[alumnos['alumnos_id']==5]

,Nombre,Email,vertical_id,alumnos_id
0,Jafet Casals,Jafet_Casals@gmail.com,55,5


In [162]:
# Volvemos a crear la conexión 

conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )

cursor = conn.cursor()

# Iteramos por las filas y metemos los datos del Df en las columnas correspondientes
for index, row in alumnos.iterrows():
    cursor.execute("""
        INSERT INTO "alumno" ("nombre", "email", "vertical_id", "alumno_id")
        VALUES (%s, %s, %s, %s)
    """, (row['Nombre'], row['Email'], row['vertical_id'], row['alumnos_id']))
conn.commit()
# cursor.close()

In [163]:
conn.close()

In [48]:
df = pd.DataFrame(columns=["vertical_id","rama", "promocion", "fecha_inicio", "campus"], data=rows)

In [49]:
df

,vertical_id,rama,promocion,fecha_inicio,campus
0,55,DS,Septiembre,2023-09-18,1
1,56,DS,Febrero,2024-02-12,1
2,57,FS,Septiembre,2023-09-18,1
3,58,FS,Febrero,2024-02-12,2


In [50]:
df.to_csv("verticales_sql.csv")

In [92]:
# Para borrar la tabla
# cursor.execute('''
#                 DROP TABLE "Profesores"
#                 ''')
# conn.commit()
# cursor.close()

,0,1,2,3,4,5
0,1,Noa Yáñez,DS,TA,Septiembre,Madrid
1,2,Saturnina Benitez,DS,TA,Septiembre,Madrid
2,3,Anna Feliu,FS,TA,Septiembre,Madrid
3,4,Rosalva Ayuso,FS,TA,Septiembre,Valencia
4,5,Ana Sofía Ferrer,FS,TA,Febrero,Valencia
5,6,Angélica Corral,FS,TA,Febrero,Madrid
6,7,Ariel Lledó,DS,TA,Septiembre,Madrid
7,8,Mario Prats,FS,LI,Febrero,Valencia
8,9,Luis Ángel Suárez,FS,LI,Septiembre,Madrid
9,10,María Dolores Diaz,DS,LI,Septiembre,Madrid


In [1]:
cursor.close()

NameError: name 'cursor' is not defined